In [1]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.io import output_notebook, curdoc
from bokeh.models import LinearAxis, Range1d
from bokeh.models import Span, Label
from bokeh.palettes import Category10

output_notebook()

curdoc().theme = 'dark_minimal'

Loading BokehJS ...

# Band structures of selected 2D lattices

## Introduction
This notebook provides a deep dive into the calculation of band structures of some commonly encountered two-dimensional (2D) lattices. The lattices we'll focus on are the square, triangular, Lieb, and honeycomb lattices. We will perform these calculations using the tight-binding approximation, which is a popular method in quantum mechanics to calculate the electronic band structure of materials.


## Square lattice
The square lattice is the simplest and most symmetric lattice in 2D. It consists of one atom per unit cell and each atom has four nearest neighbours. The Hamiltonian for such a lattice can be expressed considering equal hopping integrals t to the four nearest neighbour atoms(see the image below). The hopping integral represents the overlap of electron wavefunctions between neighboring atoms and corresponds to the energy required for an electron to 'hop' from one atom to another:
$$ H(k)=t  \exp(ik_x a)+t  \exp(-ik_x a)+t  \exp(ik_y a)+t  \exp(-ik_y a)$$

Here k represents the crystal momentum, i is the imaginary unit, and a is the lattice constant.

![square_lattice.png](square_lattice.png)

### Analytic solution

The band structure for a square lattice can be found analytically. In solid state physics, the band structure of a solid describes the range of energy levels that electrons may have within it, as well as the ranges of energy that they may not have (called band gaps).

The energy E(k) as a function of k for a square lattice is:
$$E(k)=2t \cos(k_x a)+2t \cos(k_y a)$$

Our code plots this energy band structure along a high symmetry path in the first Brillouin zone, which has a square shape for a square lattice.


In [2]:
# Define a function 'band' that creates energy band plots for different crystal lattices
def band(nkl, kpoints, ham, title, ylabel):
    # Set the number of k-points and initialize variables
    nk = 32
    n = 0
    d = np.size(ham([0.0, 0.0]), 1)
    evk = np.zeros((nk * nkl - nk - 1, d))
    lenk = np.zeros((nk * nkl - nk - 1, d))
    xticks_labels = ['Г', 'M', 'K', 'Г']

    # Initialize the figure with specific labels, range and size
    fig = figure(title=title, x_axis_label='', y_axis_label=ylabel, x_range=(-5, 99), 
                 width=800, height=400)

    # Set the x-axis ticks and labels
    fig.xaxis.ticker = [0, 31, 62, 94]
    fig.xaxis.major_label_overrides = {0: xticks_labels[0], 31: xticks_labels[1], 62: xticks_labels[2], 94: xticks_labels[3]}
    fig.yaxis.axis_label_text_font_style = "italic"

    # Calculate the energy eigenvalues for each k-point and store them in evk
    for i in range(0, nkl - 1):
        klines = np.linspace(kpoints[i], kpoints[i + 1], nk)

        for j in range(0, nk):
            mat = 0
            k = klines[j]
            mat = ham(k)
            n = i * nk + j
            evk[n - 1] = np.real(np.linalg.eigvalsh(mat))
            lenk[n - 1] = n

    # Use color palette to color lines
    colors = Category10[10]

    # Plot each band with a unique color
    for i in range(d):
        fig.line(lenk[:, i], evk[:, i], line_width=2, legend_label=f'Band {i+1}', line_color=colors[i%10])

    # Add dotted red lines at specific x-axis points
    for loc in [0, 31, 62, 94]:
        red_line = Span(location=loc, dimension='height', line_color='red', line_dash='dashed', line_width=2)
        fig.add_layout(red_line)
        
        # Optional, add labels for the lines
        label = Label(x=loc, y=5, text=xticks_labels[loc // 31], text_color='red')
        fig.add_layout(label)

    # Set the legend location and click policy
    fig.legend.location = "top_right"
    fig.legend.click_policy = "hide"

    # Automatically adjust y range to include all the data
    fig.y_range.start = np.min(evk)
    fig.y_range.end = np.max(evk)

    # Return the figure
    return fig

In [3]:
# Define the function for calculating the Hamiltonian of a 2D square lattice
def square_2d(kd, t):
    ham = np.zeros((1, 1)) + 0.j
    k = [kd[0] * 2 * np.pi, kd[1] * 2 * np.pi]
    k1 = k[0]
    k2 = k[1]
    ham[0][0] = 2 * t * np.cos(k1) + 2 * t * np.cos(k2)
    return ham

# A function to visualize the band structure for a 2D square lattice
def band_square(t):
    nkl = 4
    kpoints = [[0.00, 0.00], [0.00, 0.50], [0.50, 0.50], [0.00, 0.00]]
    ham1 = lambda k: square_2d(k, t)

    fig = band(nkl, kpoints, ham1, title='Square lattice', ylabel='Energy/t')
    show(fig)

    
band_square(1.0)



## Triangular lattice

The triangular (or hexagonal) lattice is another commonly encountered 2D lattice, especially in the study of graphene and other 2D materials. Unlike the square lattice, each site in a triangular lattice has six nearest neighbours. This results in a different Hamiltonian and a different band structure.

![triangular.png](triangular.png)

In [4]:
# Define the function for calculating the Hamiltonian of a 2D triangular lattice
def triangular_2d(kd, t):
    ham = np.zeros((1, 1)) + 1.j
    k = [kd[0] * 2 * np.pi, kd[1] * 2 * np.pi]
    k1 = k[0]
    k2 = k[1]
    ham[0][0] = np.exp(k1 * 1.j) * t + np.exp(k2 * 1.j) * t + t * np.exp((k1 - k2) * 1.j) + np.exp(-k1 * 1.j) * t + \
               np.exp(-k2 * 1.j) * t + t * np.exp((k2 - k1) * 1.j)
    return ham

# A function to visualize the band structure for a 2D triangular lattice
def band_triangular(t):
    nkl = 4
    kpoints = [[0.00, 0.00], [0.00, 0.50], [0.33, 0.66], [0.00, 0.00]]
    ham1 = lambda k: triangular_2d(k, t)

    fig = band(nkl, kpoints, ham1, title='Triangular lattice', ylabel='Energy/t')
    show(fig)

    
band_triangular(1.0)



## Honeycomb lattice

The honeycomb lattice is special as it consists of two intertwined triangular lattices. This leads to fascinating properties, as seen in graphene. The lattice can be seen as generated by two sublattices, denoted by A and B.

The Hamiltonian matrix for a honeycomb lattice is $2 \times 2$ matrix::

$$H(k)=\left(\begin{array}{cc}
0 & t( \exp(ik\cdot \delta_1)+\exp(ik\cdot \delta_2)+\exp(ik\cdot \delta_3))\\
t(\exp(-ik\cdot \delta_1)+\exp(-ik\cdot \delta_2)+\exp(-ik\cdot \delta_3)) & 0
\end{array}\right)$$
where $\delta_i$ are vectors connecting nearest-neighbour atoms.



![honeycomb.png](honeycomb.png)   

The eigenvalues of this matrix have the following analytic form:
$$E(k)=\pm \sqrt{3 + 2 \cos(k\cdot\delta_1) + 2 \cos(k\cdot\delta_2) + 2 \cos(k\cdot\delta_3)}$$

In [17]:
# Define the function for calculating the Hamiltonian of a 2D honeycomb lattice
def band_honeycomb(t):
    nkl = 4
    kpoints = [[0.00, 0.00], [0.00, 0.50], [0.33, 0.66], [0.00, 0.00]]
    ham1 = lambda k: honeycomb_2d(k, t)

    fig = band(nkl, kpoints, ham1, title='Honeycomb lattice', ylabel='Energy/t')
    show(fig)

# A function to visualize the band structure for a 2D honeycomb lattice
def honeycomb_2d(kd, t):
    ham = np.zeros((2, 2)) + 1.j
    k = [kd[0] * 2 * np.pi, kd[1] * 2 * np.pi]
    k1 = k[0]
    k2 = k[1]
    ham[0][1] = np.exp(k1 * 1.j) * t + np.exp(k2 * 1.j) * t + t
    ham[1][0] = np.exp(-k1 * 1.j) * t + np.exp(-k2 * 1.j) * t + t
    return ham


band_honeycomb(1.0)

## Lieb lattice
The Lieb lattice is the most complex case as it has 3 atoms per unit cell:  

![lieb.png](lieb.png)

In [18]:
# Define the function for calculating the Hamiltonian of a 2D lieb lattice
def lieb_2d(kd, t):
    ham = np.zeros((3, 3)) + 0.j
    k = [kd[0] * 2 * np.pi, kd[1] * 2 * np.pi]
    k1 = k[0]
    k2 = k[1]
    ham[0][1] = t * (1 + np.exp(-k1 * 1.j))
    ham[0][2] = t * (1 + np.exp(-k2 * 1.j))
    ham[1][0] = ham[0][1].conj()  # The conjugate of ham[0][1]
    ham[2][0] = ham[0][2].conj()  # The conjugate of ham[0][2]
    return ham


# A function to visualize the band structure for a 2D lieb lattice
def band_lieb(t):
    nkl = 4
    kpoints = [[0.00, 0.00], [0.00, 0.50], [0.50, 0.50], [0.00, 0.00]]
    ham1 = lambda k: lieb_2d(k, t)

    fig = band(nkl, kpoints, ham1, title='Lieb lattice', ylabel='Energy/t')
    show(fig)




band_lieb(1.0)

# Questions

- Modify the code such that you can introduce unequal hopping integrals (e.g. $t_1 \ne t_2$ for the vertical and horizontal bonds in square lattice) and investigate their effect on the band structures.
- Modify the code to introduce unequal hopping integrals (e.g. t_1 ≠ t_2 for the vertical and horizontal bonds in the square lattice). How does this affect the band structures?
- How does the number of nearest neighbours affect the complexity of the Hamiltonian and consequently the band structure?
- How would you modify the Hamiltonian to account for next nearest neighbours interactions? Try implementing this in the code. What changes do you observe in the band structure?
- What happens to the band structure if you introduce a periodic potential to the lattice?